# Prova 3

Instructions:
- Exercises are grouped by topic and ordered by increasing difficulty within each topic.
- Submit your solutions in a well-organized Jupyter notebook.
- You are responsible for creating robust test cases for each question. Ensure your tests cover all possible cases, especially edge cases (e.g., with the largest possible N ).
- Verify that all your solutions meet the specified time constraints. Unless otherwise stated, each question has a 1-second time limit. Some of the later questions may have different limits, which will be indicated.
- Be sure that the inputs (which will be created by you) match the specifications, and your solutions meets the required time limit.
- Presentation matters: make sure your notebook is clean, well-structured, and that both solutions and test cases are clearly labeled.

## Distinguishing Unitaries and States

**Exercise 1.** You are given an operation that implements a single-qubit unitary transformation: either the H gate or the X gate. The operation will have Adjoint and Controlled variants defined. Your task is to perform necessary operations and measurements to figure out which unitary it was and to return 0 if it was the H gate or 1 if it was the X gate. You are allowed to apply the given operation and its adjoint/controlled variants at most twice. You have to implement an operation which takes a single-qubit operation as an input and returns an integer.

In [2]:
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import GlobalPhaseGate, PhaseOracle
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler

from sympy.parsing.sympy_parser import parse_expr
from sympy.logic.inference import satisfiable

import numpy as np